In [ ]:
# ignore this section, written to be short as possible
function nb.JoinCsv { 
    [Alias('Csv', 'UL')]
    [cmdletbinding()]
    param(
        <# prefix string #>[Alias('op')][Parameter(Position=0)]$Prefix = '',
        <# suffix string #> [Alias('os')][Parameter(Position=1)]$Suffix = '',
        <# suffix string #> [Alias('Sep')][Parameter()]$Separator = ', ',
        <# suffix property #> [Alias('Prop', 'P')][Parameter()]$PropertyName = $Null,
        <# Actual Pipe #>[Parameter(ValueFromPipeline, Mandatory)][object[]]$InputObject,
        [switch]$Sort, [switch]$Unique, [int]$Pad = 0
    )
    begin {
        $items = [System.Collections.Generic.list[object]]::new()
    }
    process {
        foreach($cur in $InputObject) { $Items.add($cur)}
    } 
    end {
        $isUL = $PSCmdlet.MyInvocation.InvocationName -eq 'UL'

        $padding = "`n" * $Pad -join ''
        $Prefix = $Padding + $Prefix
        $Suffix = $Suffix + $Padding

        
        if($False -and 'original UL' -and $IsUL) {             
                $Prefix = $Prefix + "`n - "
                $Suffix = "`n - " + $Suffix
                $Separator = "`n - "
        }
        if($IsUL) {             
            if( (-not $Separator) -or $Separator -eq ', ') {
                $Separator = ' - '    
            }
            $Prefix = $Prefix + "`n${Separator}"
            $Suffix = "`n${Separator}" + $Suffix
            $Separator = "`n${Separator}"
        }
        
        $splat = @{  'sep' = $Separator ; 'op' =  $Prefix ; 'os' = $Suffix }
        $IsRawText = $items[0] -is 'string'
        write-debug $IsRawText
        
        $filtered = if($Unique) { $Items | Sort -Unique }
        elseif($Sort) { $Items | sort }
        else { $items } 

        if($PropertyName -and (! $IsRawText)) { 
            $filtered | Join-String @splat -Prop $PropertyName
            # $Items | Join-String -sep ', ' -op:$Prefix -os $Suffix -Prop $PropertyName
        } else {
            $filtered | Join-String @splat
            # $Items | Join-String -sep ', ' -op:$Prefix -os $Suffix
        }
    }
}

('last executed: {0}' -f @((get-date).tostring('u')))
Import-Module NameIt -ea stop -passthru | Csv -Prop { $_.Name, $_.Version } -op 'modules = '
Import-Module NameIt -ea stop -passthru | Ul -Prop { $_.Name, $_.Version } -op 'modules = '
$PSVersionTable | join-string -op 'Pwsh: ' -Property PSVersion
Get-Command -m NameIT | Csv -prop Name -op 'NameIt Commands = '
Get-Command -m NameIT | ul -prop Name -op 'NameIt Commands = '



last executed: 2022-07-04 10:55:26Z
modules = NameIt 2.3.2
modules = NameIt 2.3.2
Pwsh: 7.2.4
NameIt Commands = Invoke-Generate, New-NameItTemplate
NameIt Commands = Invoke-Generate, New-NameItTemplate


<a name='nb-top' href='#'>Top</a>
### Using [github.com/dfinke/NameIt](https://github.com/dfinke/NameIt)
<!-- - <a href='#nb-item-1'>Empty File</a>
- [Load Column, Row, hashtable](https://github.com/dfinke/ImportExcel/blob/master/FAQ/How%20to%20Read%20an%20Existing%20Excel%20File.md#other-common-operations) -->

In [ ]:
$Templates = @(
    '[name], [color]'
    '[address]'
    '####'
    '??-###-####-?##'
)
$Templates | %{ 
    foreach($i in 0..3) { 
    @(
        $_.PadLeft(20)
        ig $_
    ) -join ' = '
    }
}


     [name], [color] = name, gray
     [name], [color] = name, orange
     [name], [color] = name, magenta
     [name], [color] = name, beige
           [address] = 59994 Ocutudzaza Pr
           [address] = 94400 Wobjakuney Corners
           [address] = 409 Nihwadqoulej Course
           [address] = 317 Bavomor Un
                #### = 7587
                #### = 3341
                #### = 2702
                #### = 4446
     ??-###-####-?## = se-275-7150-x79
     ??-###-####-?## = yu-134-0947-u11
     ??-###-####-?## = ub-371-1236-n05
     ??-###-####-?## = ha-771-4884-i56

 - [name], [color]
 - name, cyan
 - name, orange
 - name, beige
 - name, blue
 - [address]
 - 140 Yuwuxoqnuni Curve
 - 9161 Idro Sq
 - 83628 Yivwuvomri Is
 - 48402 Uhukeclisduc Run
 - ####
 - 6461
 - 9313
 - 5021
 - 6355
 - ??-###-####-?##
 - yk-763-8462-x23
 - io-227-8089-f33
 - xe-255-0871-w36
 - pb-901-2724-k46


In [ ]:
'aaz'
$Templates | %{ 
    $_ | Csv 'Template = '
    @(foreach($i in 0..3) { 
        @(
            # $_.PadLeft(20)
            ig $_ 
        ) #| csv 
    }) | Ul
} #| Ul

aaz
Template = [name], [color]

 - name, khaki
 - name, yellow
 - name, violet
 - name, cyan
Template = [address]

 - 426 Guvepvi Dr
 - 3870 Taqlew Byp
 - 34570 Tuap Overpass
 - 59709 Xuxgaiwvoik Parkways
Template = ####

 - 8352
 - 1887
 - 9321
 - 5553
Template = ??-###-####-?##

 - cl-846-6962-o76
 - ht-698-6665-c57
 - cq-921-7531-s96
 - qp-336-6788-e70


In [ ]:

#from the docs
Invoke-Generate -alpha 'abc' -count 3 | Csv 'abc: ' -pad 1


Invoke-Generate -count 5 "[synonym cafe]_[syllable][syllable]" | csv -op 'syllable: ' -pad 1 -sep "`n- "


abc: abababca, bccccbba, aabcbcab


syllable: eatingplace_cizcor
- eatingplace_begya
- coffeehouse_nixaw
- coffeebar_inal
- eatinghouse_xeppeg



<a href='#nb-top'>Go Top</a>
- [Plot](https://github.com/dfinke/ImportExcel/blob/master/Examples/Charts/plot.ps1)

<h4 id='nb-bottom'>See More</h4>
See more:
- Works well with [dfinke/ImportExcel](https://github.com/dfinke/ImportExcel)
- repo: [dfinke/NameIt](https://github.com/dfinke/NameIt)
  